In [ ]:
#아래의 URL에서 cohere key 받기
#https://dashboard.cohere.com/api-keys

In [1]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.5/252.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.0 MB/s eta 0:00:00


In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [4]:
import os
import openai
from getpass import getpass

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

In [5]:
from google.colab import files
uploaded = files.upload()

Saving 1706.03762v7.pdf to 1706.03762v7.pdf


In [7]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 3.9 MB/s eta 0:00:00


In [8]:
from langchain.document_loaders import PyPDFLoader

# PDF 가져오기
loaders = [PyPDFLoader(filename) for filename in uploaded.keys()]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
# Splitter를 이용한 청킹
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from openai import OpenAI
api_key=""

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 10
)

#splitter를 이용한 문서 청킹
splits = text_splitter.split_documents(docs)
print(splits)

In [10]:
#임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)


<ipython-input-10-7ad24faf6ccc>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)


In [12]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.6 MB/s eta 0:00:00


In [14]:
!pip install faiss-gpu-cu12 --extra-index-url=https://pypi.nvidia.com --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 203.9 MB/s eta 0:00:00


In [15]:
vectorstore = FAISS.from_documents(splits, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 13})

In [16]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-4", api_key=api_key)

<ipython-input-16-7091c6e655b2>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", api_key=api_key)


In [17]:
%%time
qa1 = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever)

query = "트랜스포머란?"
print(qa1.run(query=query))

<timed exec>:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.


트랜스포머는 복잡한 순환 또는 합성곱 신경망을 기반으로 하는 시퀀스 변환 모델을 대체하는 새로운 네트워크 아키텍처입니다. 이는 순환과 합성곱을 생략하고 전적으로 attention 메커니즘에만 의존합니다. 이로 인해 훨씬 더 많은 병렬화가 가능해지며, 학습 시간이 크게 단축됩니다. 번역 작업에서는 이전의 순환 또는 합성곱 레이어를 기반으로 하는 아키텍처보다 훨씬 빠르게 트랜스포머를 학습할 수 있습니다. WMT 2014 영어-독일어 및 WMT 2014 영어-프랑스어 번역 작업에서 최신 기술을 달성했습니다.
CPU times: user 143 ms, sys: 15.7 ms, total: 158 ms
Wall time: 15.2 s


In [20]:
os.environ["COHERE_API_KEY"] = getpass("Cohere API Key:")
compressor = CohereRerank()
# ContextualCompressionRetriever: 관련 문서 검색
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(query)
print(compressed_docs)

Cohere API Key:··········


<ipython-input-20-7929bc4aa28b>:2: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank()
<ipython-input-20-7929bc4aa28b>:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  compressed_docs = compression_retriever.get_relevant_documents(query)


[Document(metadata={'source': '1706.03762v7.pdf', 'page': 1, 'page_label': '2', 'relevance_score': 0.14199378}, page_content='Most competitive neural sequence transduction models have an encoder-decoder structure [5, 2, 35].\nHere, the encoder maps an input sequence of symbol representations (x1, ..., xn) to a sequence\nof continuous representations z = (z1, ..., zn). Given z, the decoder then generates an output\nsequence (y1, ..., ym) of symbols one element at a time. At each step the model is auto-regressive\n[10], consuming the previously generated symbols as additional input when generating the next.\n2'), Document(metadata={'source': '1706.03762v7.pdf', 'page': 8, 'page_label': '9', 'relevance_score': 0.11357959}, page_content='Table 3: Variations on the Transformer architecture. Unlisted values are identical to those of the base\nmodel. All metrics are on the English-to-German translation development set, newstest2013. Listed\nperplexities are per-wordpiece, according to our byt

In [21]:
qa2 = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=compression_retriever )

In [22]:
%%time
#
print(qa2.run(query=query))

트랜스포머는 인공지능 분야에서 사용되는 신경망 아키텍처의 일종으로, 텍스트나 시퀀스 데이터를 다루는 데 주로 사용됩니다. 이는 인코더-디코더 구조를 가지며, 인코더는 입력 시퀀스를 연속적인 표현으로 매핑하고, 디코더는 이 연속적인 표현을 기반으로 출력 시퀀스를 생성합니다. 이 각각의 단계에서 모델은 자동 회귀적인 방법을 사용하여 이전에 생성된 심볼을 다음 심볼 생성 시의 추가 입력으로 사용합니다. 트랜스포머는 기본적으로 각 위치의 입력이 전체 시퀀스에 동시에 접근할 수 있도록 설계되어 있어, 전통적인 순차적인 처리 방식보다 훨씬 빠르게 처리할 수 있습니다. 트랜스포머는 번역, 텍스트 요약, 감성 분석 등 다양한 자연어 처리 작업에 사용되며, 특히 번역 작업에서 뛰어난 성능을 보여주었습니다.
CPU times: user 139 ms, sys: 20.3 ms, total: 159 ms
Wall time: 21.1 s


In [ ]:
한국형 스마트팜은 우리나라의 농업환경과 현장 상황에 적합한 기술을 개발하여 농업선진국과 경쟁할 수 있는 독자적인 농업생산기술을 확보하는 것을 목표로 합니다.
이는 사물인터넷, 빅데이터, 인공지능, 로봇 등을 활용하여 농산물의 생육환경을 최적상태로 관리하고, 노동력 절감과 생산성 향상을 구현하는 효율적인 농업형태입니다.
한국형 스마트팜 기술개발 프로젝트는 기술수준별로 스마트팜 모델을 3가지 단계로 구분하여 개발을 추진하고 있습니다.
이 단계는 편리성 증진을 위한 1세대, 생산성 향상을 위한 2세대, 그리고 글로벌 산업화를 위한 3세대로 나뉘어져 있습니다.